In [1]:

%load_ext autoreload
%autoreload 2
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)



import numpy as np
import pickle

import os
import sys

module_path = os.path.abspath(os.path.join('../src'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from tqdm import tqdm
from utils import *

C:\Users\Ignatius Ting\AppData\Local\Temp\ipykernel_23968\1375793260.py:7: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


In [2]:
def combine_data(folder_path):   
    # Get a list of all CSV files in the folder
    csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

    # Initialize an empty list to store dataframes
    dfs = []

    # Loop through each CSV file and read it into a dataframe
    for csv_file in csv_files:
        csv_path = os.path.join(folder_path, csv_file)
        df = pd.read_csv(csv_path)
        dfs.append(df)

    # Concatenate all dataframes into a single dataframe
    df_combined = pd.concat(dfs, ignore_index=True)
    return df_combined


# path_1mm = '../data/processed/1mm_match/'
# df_1mm = combine_data(path_1mm)

# path_1mm_trim = '../data/processed/1mm_trim_match/'
# df_1mm_trim = combine_data(path_1mm_trim)

# path_1m = '../data/processed/1m_match/'
# df_1m = combine_data(path_1m)

### process single hop data


In [3]:
df = combine_data('../data/processed/1m_match/')

In [4]:
from langdetect import detect
def detect_language(text):
    texts = text.split('\n')
    try:
        detected_languages = []
        for x in texts:
            lang = detect(x)
            detected_languages.append(lang)

        return list(set(detected_languages))

    except:
        return 'unknown'  # Return "Unknown" if language detection failsection fails


In [5]:
print(df.shape)
df = df.drop_duplicates('query', keep = 'first')
print(df.shape)
df = df.drop_duplicates('url', keep = 'first')
print(df.shape)
# Apply the detect_language function to the 'TextColumn'
df['langs'] = df.result.apply(lambda x: detect_language(x))
df['temp'] = df['langs'].apply(len)
df = df[df.temp == 1]
df['langs'] = df.langs.apply(lambda x: x[0])
df = df[df.langs == 'en']
print(df.shape)

k = 2500
p = 1000
df['result_len'] = df.result.apply(len)
df = df[(df.result_len < k) & (df.result_len > p)].reset_index(drop = True)

(94939, 6)
(94938, 6)
(33891, 6)
(32316, 8)


In [6]:
import pickle
with open('../data/all_dicts.pkl', 'rb') as f:
    all_dicts = pickle.load(f)
desc_dict = all_dicts['desc_dict']


In [7]:
df['source_text'] = df['source_cui'].apply(lambda x: desc_dict[x])
df['cui_2_text'] = df['cui_2'].apply(lambda x: desc_dict[x])


In [8]:
df.to_csv('../data/final/1hop.csv', index = False)

### process 2hop data

In [9]:
df2 = combine_data('../data/processed/1mm_final_match/')

In [10]:
print(df2.shape)
df2 = df2.drop_duplicates('query', keep = 'first')
print(df2.shape)
df2 = df2.drop_duplicates('url', keep = 'first')
print(df2.shape)

(98470, 8)
(97003, 8)
(37626, 8)


In [11]:

# Apply the detect_language function to the 'TextColumn'
df2['langs'] = df2.result.apply(lambda x: detect_language(x))
df2['temp'] = df2['langs'].apply(len)
df2 = df2[df2.temp == 1]
df2['langs'] = df2.langs.apply(lambda x: x[0])
df2 = df2[df2.langs == 'en']
print(df2.shape)
df2['result_len'] = df2.result.apply(len)
df2 = df2[(df2.result_len < k) & (df2.result_len > p)].reset_index(drop = True)
print(df2.shape)

(35929, 10)
(28553, 11)


In [12]:
df2['source_text'] = df2['source_cui'].apply(lambda x: desc_dict[x])
df2['cui_2_text'] = df2['cui_2'].apply(lambda x: desc_dict[x])
df2['cui_3_text'] = df2['cui_3'].apply(lambda x: desc_dict[x])

In [13]:
df2['c1'] = df2.apply(lambda x: x['source_text'].lower() in x['result'].lower() , axis=1)
df2['c2'] = df2.apply(lambda x: x['cui_2_text'].lower()  in x['result'].lower() , axis=1)
df2['c3'] = df2.apply(lambda x: x['cui_3_text'].lower()  in x['result'].lower() , axis=1)
df2['c1_c2'] = df2.apply(lambda x: x['source_text'].lower()  in x['result'].lower()  and x['cui_2_text'].lower()  in x['result'].lower() , axis=1)
df2['c2_c3'] = df2.apply(lambda x: x['cui_2_text'].lower()  in x['result'].lower()  and x['cui_3_text'].lower()  in x['result'].lower() , axis=1)
df2['c1_c2_c3'] = df2.apply(lambda x: x['source_text'].lower()  in x['result'].lower()  \
                          and x['cui_2_text'].lower()  in x['result'].lower()  and x['cui_3_text'].lower()  in x['result'].lower() , axis=1)


In [14]:
df2.c1.value_counts()

False    25352
True     3201 
Name: c1, dtype: int64

In [15]:
df2.c2.value_counts()

False    24249
True     4304 
Name: c2, dtype: int64

In [16]:
df2.c3.value_counts()

False    25350
True     3203 
Name: c3, dtype: int64

In [17]:
df2.c1_c2.value_counts()

False    27786
True     767  
Name: c1_c2, dtype: int64

In [18]:
df2.c1_c2_c3.value_counts()

False    28339
True     214  
Name: c1_c2_c3, dtype: int64

In [19]:
df2[df2.c1_c2_c3 == True][['query','result']].sample(1)

,query,result
1637,Autosomal recessive hereditary spastic paraplegia has finding site Spinal cord finding site of Hereditary spastic paraplegia,"The detailed neuropathological features of patients with autosomal recessive hereditary spastic paraplegia with a thin corpus callosum (TCC) and SPG11 mutations are poorly understood, as only a few autopsies have been reported. Herein, we describe the clinicopathological findings of a patient with this disease who received long-term care at our medical facility.\nA Japanese man exhibited a mild developmental delay in early childhood and intellectual disability, followed by the appearance of a spastic gait by age 13. At the age of 25 years, he became bedridden and needed a ventilator. Genetic analysis revealed a homozygous splice site variant in the SPG11 gene (c. 4162-2A > G) after the provision of genetic counselling and acquisition of informed consent from his parents. He died of pneumonia at the age of 44. His brain weighed 967 g and was characterized by a TCC, and his spinal cord was flattened. Microscopically, degeneration was observed in the posterior spinocerebellar tract, the gracile fasciculus, and the posterior column in addition to the corticospinal tract. Marked neuronal loss and gliosis were observed in the anterior horn, Clarke's column, and hypoglossal and facial nuclei. Various types of neurons, in addition to motor neurons, showed coarse eosinophilic granules that were immunoreactive for p62. The loss of pigmented neurons with gliosis was apparent in both the substantia nigra and locus coeruleus. Lateral geniculate body degeneration was a characteristic feature of this patient. Furthermore, peripheral Lewy body-related α-synucleinopathy and scattered α-synuclein-immunoreactive neurites in the locus coeruleus and reticular formation of the brainstem were observed.\nIn patients with hereditary spastic paraplegia with SPG11 mutations, a variety of clinical phenotypes develop due to widespread lesions containing p62-immunoreactive neuronal cytoplasmic inclusions. We herein report the lateral geniculate body as another degenerative site related to SPG11-related pathologies that should be studied in future investigations."


In [20]:
df2 = df2.drop(['langs',
'temp',
'result_len',
'c1', 'c2', 'c3', 'c1_c2', 'c2_c3',
'c1_c2_c3'], axis = 1)

In [21]:
df2.to_csv('../data/final/2hop.csv', index = False)

### Preprocess 3hop

In [22]:
df_3 = combine_data('../data/processed/3hop_match/')

In [23]:
df_3.sample(1)[['relation','relation_2','relation_3','query']]

,relation,relation_2,relation_3,query
2990,has finding site,is not primary anatomic site of disease,may be treated by,"Narrowing of bone has finding site Bone, skeletal is not primary anatomic site of disease Infarction;myocardial may be treated by Vorapaxar"


In [24]:
print(df_3.shape)
df_3 = df_3.drop_duplicates('query', keep = 'first')
print(df_3.shape)
df_3 = df_3.drop_duplicates('url', keep = 'first')
print(df_3.shape)
# Apply the detect_language function to the 'TextColumn'
df_3['langs'] = df_3.result.apply(lambda x: detect_language(x))
df_3['temp'] = df_3['langs'].apply(len)
df_3 = df_3[df_3.temp == 1]
df_3['langs'] = df_3.langs.apply(lambda x: x[0])
df_3 = df_3[df_3.langs == 'en']
print(df_3.shape)

k = 2500
p = 1000
df_3['result_len'] = df_3.result.apply(len)
df_3 = df_3[(df_3.result_len < k) & (df_3.result_len > p)].reset_index(drop = True)

(9800, 10)
(9799, 10)
(3020, 10)
(2887, 12)


In [25]:
df_3['source_text'] = df_3['source_cui'].apply(lambda x: desc_dict[x])
df_3['cui_2_text'] = df_3['cui_2'].apply(lambda x: desc_dict[x])
df_3['cui_3_text'] = df_3['cui_3'].apply(lambda x: desc_dict[x])
df_3['cui_4_text'] = df_3['cui_4'].apply(lambda x: desc_dict[x])

In [26]:
print(df_3.shape)
df_3['langs'] = df_3.result.apply(lambda x: detect_language(x))
df_3['temp'] = df_3['langs'].apply(len)
df_3 = df_3[df_3.temp == 1]
df_3['langs'] = df_3.langs.apply(lambda x: x[0])
df_3 = df_3[df_3.langs == 'en']
print(df_3.shape)
df_3['result_len'] = df_3.result.apply(len)
df_3 = df_3[(df_3.result_len < k) & (df_3.result_len > p)].reset_index(drop = True)
print(df_3.shape)

(2224, 17)
(2222, 17)
(2222, 17)


In [27]:
df_3['c1_c2_c3_c4'] = df_3.apply(lambda x: x['source_text'].lower()  in x['result'].lower()  \
                          and x['cui_2_text'].lower()  in x['result'].lower() \
                            and x['cui_3_text'].lower()  in x['result'].lower()\
                                 and x['cui_4_text'].lower() in x['result'].lower() , axis=1)

In [28]:
df_3.c1_c2_c3_c4.value_counts()

False    2219
True     3   
Name: c1_c2_c3_c4, dtype: int64

In [29]:
df_3[df_3.c1_c2_c3_c4 == True][['query','result']]

,query,result
108,Potassium iodide may treat sporotrichosis may be treated by amphotericin b may treat histoplasmosis,"Successful chemotherapy of the systemic mycoses now covers a span of more than 75 years and dates to the first reported use of potassium iodide for treatment of sporotrichosis. The second drug with efficacy was stilbamidine, and its currently available successor, hydroxystilbamidine isethionate, still has a role in therapy of some patients with nonprogressive blastomycosis of the skin. The introduction in 1957 of amphotericin B marked the first time there was an effective agent for such diseases as cryptococcosis, histoplasmosis, candidosis, and with lesser success, for coccidioidomycosis, mucormycosis, and aspergillosis. However, amphotericin B is nephrotoxic, depresses bone marrow (especially erythropoeisis), and, if patients are not monitored and controlled closely, the drug produces hypokalemic muscle weakness and cardiotoxicity. Flucytosine has a narrower spectrum of activity (cryptococcosis, candidosis, cladosporiosis, and chromomycosis) but a preferable route of administration (oral). Newer agents presently available are miconazole and clotrimazole; the latter is for topical use only."
319,Antiviral agents therapeutic class of Velpatasvir may treat hepatitis C may be treated by Paritaprevir,"The intestinal microbiota plays a fundamental role in physiological homeostasis as well as in pathologic conditions. Hepatitis C virus is the leading cause of chronic liver diseases worldwide. The treatment of this infection has been revolutionized by the availability of direct-acting antiviral agents which guarantee a high rate (about 95%) of viral clearance. Few studies have assessed the change in the gut microbiota of patients treated with direct-acting antiviral agents against HCV, and many aspects still need to be clarified. The aim of the study was to evaluate the effects of antiviral therapy on gut microbiota. We enrolled patients with HCV-related chronic liver disease attending the Infectious Diseases Unit of the A.O.U. Federico II of Naples from January 2017 to March 2018 and treated with DAAs. For each patient, a fecal sample was collected and analyzed for the assessment of microbial diversity before the start of therapy and by SVR12 time. We excluded patients who had received antibiotics in the previous 6 months. Twelve patients were enrolled (6 male, 8 genotype 1 (1 subtype 1a), 4 genotype 2). Fibrosis scores were F0 in 1 patient, F2 in 1 patient, F3 in 4 patients and cirrhosis in the remaining 6 (all in Child-Pugh class A). All were treated with DAAs for 12 weeks (5 with Paritaprevir-Ombitasvir-Ritonavir-Dasabuvir, 3 with Sofosbuvir-Ledipasvir, 1 with Sofosbuvir-Ribavirin, 1 with Sofosbuvir-Daclatasvir, 1 with Sofosbuvir-Velpatasvir) and 100% achieved SVR12. In all patients, we observed a trend in reduction of potentially pathogenic microorganisms (i.e., Enterobacteriaceae). Furthermore, a trend of increase in α-diversity was observed in patients by SVR12 compared to baseline. This trend was markedly more evident in patients without liver cirrhosis than in those with cirrhosis. Our study shows that viral eradication obtained with DAA is associated with a trend in restoring the heterogeneity of α-diversity and in reducing the percentage of potentially pathogenic microbial species, although this benefit is less evident in patients with cirrhosis. Further studies with larger sample size are needed to confirm these data."
806,aspergillosis may be treated by amphotericin b may prevent Mycosis may be prevented by Isavuconazole,"Invasive fungal diseases (IFD) are a major global public health concern. The incidence of IFD has increased the demand for antifungal agents. Isavuconazole (ISA) is a new triazole antifungal agent that has shown promising efficacy in the prophylaxis and treatment of invasive fungal diseases. The aim of this review is to summarize the recent real-world experiences of using ISA for the t

In [30]:
df_3 = df_3.drop(['langs',
                  'temp',
                  'result_len',
                  'c1_c2_c3_c4'], axis = 1)

In [31]:
df_3.to_csv('../data/final/3hop.csv', index = False)

In [32]:
df = pd.concat([df,df2,df_3], ignore_index = True)

In [33]:
df.columns

Index(['source_cui', 'relation 1', 'cui_2', 'query', 'url', 'result', 'langs',
       'temp', 'result_len', 'source_text', 'cui_2_text', 'relation 2',
       'cui_3', 'cui_3_text', 'relation', 'relation_2', 'relation_3', 'cui_4',
       'cui_4_text'],
      dtype='object')

In [34]:
import unidecode 
def get_point(n):
    n = n.strip()
    n = n.replace('(', '')
    n = n.replace('\"', '')
    n = n.replace(')', '')
    n = n.replace(',', ' ')
    n = n.replace('_', ' ')
    n = n.replace('  ',' ')
    n = unidecode.unidecode(n)

    return n

df['source_clean'] = df.source_text.apply(lambda x: get_point(x))
df['cui_2_clean'] = df.cui_2_text.apply(lambda x: get_point(x))
df['cui_3_clean'] = df.cui_3_text.apply(lambda x: get_point(x) if not pd.isna(x) else None)
df['cui_4_clean'] = df.cui_4_text.apply(lambda x: get_point(x) if not pd.isna(x) else None)

In [35]:
df['relation_1'] = df.apply(lambda x: x['relation 1'] if pd.isna(x['relation'])\
                            else x['relation'], axis = 1)
df['relation_2'] = df.apply(lambda x: x['relation 2'] if pd.isna(x['relation_2'])\
                            else x['relation_2'], axis = 1)

In [36]:
df['rel_1_clean'] = df.relation_1.apply(lambda x: get_point(x))
df['rel_2_clean'] = df.relation_2.apply(lambda x: get_point(x) if not pd.isna(x) else None)
df['rel_3_clean'] = df.relation_3.apply(lambda x: get_point(x) if not pd.isna(x) else None)

In [37]:
pd.isna(np.nan)

True

In [38]:
def combine_entities(cui1,cui2,cui3,cui4):
    entities = [cui1,cui2,cui3,cui4]
    entities = [x for x in entities if not pd.isna(x)]
    return str(entities)
df['combined_entities'] = df.apply(lambda row: combine_entities(row['source_clean'],
                                                                row['cui_2_clean'],
                                                                row['cui_3_clean'],
                                                                row['cui_4_clean']), axis = 1)

In [39]:
df['source_t5'] = df.apply(lambda x: f"{x['combined_entities']} </s> {x['result']}", axis = 1)

In [40]:
def get_target(x):
    if pd.isna(x['rel_2_clean']):
        target = f"['{x['source_clean']}', '{x['rel_1_clean']}', '{x['cui_2_clean']}']"
    elif pd.isna(x['rel_3_clean']) and not pd.isna(x['rel_2_clean']):
        target = f"['{x['source_clean']}', '{x['rel_1_clean']}', '{x['cui_2_clean']}'], ['{x['cui_2_clean']}', '{x['rel_2_clean']}', '{x['cui_3_clean']}']"
    else:
        target = f"['{x['source_clean']}', '{x['rel_1_clean']}', '{x['cui_2_clean']}'], ['{x['cui_2_clean']}', '{x['rel_2_clean']}', '{x['cui_3_clean']}'], ['{x['cui_3_clean']}', '{x['rel_3_clean']}', '{x['cui_4_clean']}']"

    return target



In [41]:
df['target_t5'] = df.apply(lambda x: get_target(x), axis = 1)

In [42]:
df['key_id'] = list(range(df.shape[0]))

In [43]:
def get_target_enc(x):
    if pd.isna(x['rel_2_clean']):
        target = tuple([x['rel_1_clean']])
    elif pd.isna(x['rel_3_clean']) and not pd.isna(x['rel_2_clean']):
        target = (x['rel_1_clean'], x['rel_2_clean'])
    else:
        target = (x['rel_1_clean'], x['rel_2_clean'], x['rel_3_clean'])
    return target

df['target_enc'] = df.apply(lambda x: get_target_enc(x), axis = 1)

In [44]:
df_hold = df.copy()

In [45]:
# get adequate class distribution
temp = df.target_enc.value_counts()
targets = temp[temp > 4].index.tolist()
print(df.shape)
df = df[df.target_enc.isin(targets)].reset_index(drop = True)
print(df.shape)

(56598, 32)
(53888, 32)


In [46]:
df.rel_3_clean.isna().value_counts()

True     52618
False    1270 
Name: rel_3_clean, dtype: int64

In [47]:
def create_triples(row):
    triples = []
    
    for i in range(1,4):
        if i == 1:
            source_col = row[f'source_text']
        else:
            source_col = row[f'cui_{i}_text']
        cui_col = row[f'cui_{i+1}_text']
        rel_col = row[f'rel_{i}_clean']
        
        if not pd.isna(rel_col):
            triples.append((source_col, rel_col, cui_col))
    
    return triples

df['triples'] = df.apply(create_triples, axis=1)
df['triples'] = df['triples'].apply(lambda x: [triple for triple in x if triple])  # Remove empty lists
df['triples'] = df['triples'].apply(lambda x: tuple(x))
# print(df[['source_text', 'triples']])

In [48]:
df['explode_key'] = df['triples'].copy()
df_rel = df.explode('explode_key')
for x in df_rel.explode_key:
    assert len(x) == 3

In [49]:
df_rel['target_rel'] = df_rel.explode_key.apply(lambda x: x[1])
df_rel['entities'] = df_rel.explode_key.apply(lambda x: [x[0],x[2]])
df_rel['feature_rel'] = df_rel.apply(lambda x: f"{x['entities']} [SEP] {x['result']}", axis = 1)


temp = df_rel.target_rel.value_counts()
targets = temp[temp > 4].index.tolist()
print(df_rel.shape)
# get triples of relations that fall under threshold
triples_rm = df_rel[~df_rel.target_rel.isin(targets)].triples.unique()
# # Convert the inner lists to tuples
# unique_tuples = {tuple(inner_list) for sublist in triples_rm for inner_list in sublist}

# # Convert the unique tuples back to lists
# triples_rm = [list(inner_tuple) for inner_tuple in unique_tuples]
df_rel = df_rel[~df_rel.triples.isin(triples_rm)]
print(df_rel.shape)


(83411, 37)
(83411, 37)


In [50]:
# we don't want to spread instances of the same triple across train/val/test
# take one instance of each triple set, and then do split
# then we can rejoin the individual rel targets after
df_rel_split = df_rel.drop_duplicates('triples', keep = 'first')

In [51]:
from sklearn.model_selection import train_test_split
def custom_train_test_validation_split(df, train_ratio=0.6, test_ratio=0.2, validation_ratio=0.2, random_state=None):
    # First, split the data into training and temporary (test + validation) sets
    df_train, df_temp = train_test_split(df, test_size=(test_ratio + validation_ratio), random_state=random_state, stratify=df['target_enc'])
    
    # Then, split the temporary set into test and validation sets
    relative_validation_ratio = validation_ratio / (test_ratio + validation_ratio)
    df_test, df_val = train_test_split(df_temp, test_size=relative_validation_ratio, random_state=random_state, stratify=df_temp['target_enc'])
    
    return df_train, df_test, df_val

rel_train, rel_test, rel_val = custom_train_test_validation_split(df = df_rel_split)

In [52]:
rel_train = df_rel.merge(rel_train[['triples']], on = 'triples', how = 'inner')
rel_test = df_rel.merge(rel_test[['triples']], on = 'triples', how = 'inner')
rel_val = df_rel.merge(rel_val[['triples']], on = 'triples', how = 'inner')

In [53]:
rel_train.shape

(50053, 37)

In [54]:
rel_test.shape

(16686, 37)

In [55]:
rel_val.shape

(16672, 37)

In [56]:
import json
import os

def write_df_to_json(dataframe, filename, data_dir='../data/', encoding='utf-8'):
    file_path = os.path.join(data_dir, filename)

    with open(file_path, 'w', encoding=encoding) as json_file:
        # Iterate over the rows of the DataFrame
        for _, row in dataframe.iterrows():
            # Convert the row to a dictionary and write it to the file
            json.dump(row.to_dict(), json_file, ensure_ascii=False)
            # Add a newline character to separate the JSON objects
            json_file.write('\n')

### Write encoder models

In [57]:
# Example usage:
write_df_to_json(rel_train[['feature_rel','target_rel']], 'rel_train.json', data_dir = '../data/final/')
write_df_to_json(rel_val[['feature_rel','target_rel']], 'rel_val.json', data_dir = '../data/final/')
write_df_to_json(rel_test[['feature_rel','target_rel']], 'rel_test.json', data_dir = '../data/final/')

write_df_to_json(rel_train, 'full_rel_train.json', data_dir = '../data/final/')
write_df_to_json(rel_val, 'full_val.json', data_dir = '../data/final/')
write_df_to_json(rel_test, 'full_test.json', data_dir = '../data/final/')

In [65]:
train_keys = rel_train.key_id.unique()
val_keys = rel_val.key_id.unique()
test_keys = rel_val.key_id.unique()

t5_train = df_hold[df_hold.key_id.isin(train_keys)]
t5_test = df_hold[df_hold.key_id.isin(test_keys)]
t5_val = df_hold[df_hold.key_id.isin(val_keys)]

In [69]:
t5_train.columns

Index(['source_cui', 'relation 1', 'cui_2', 'query', 'url', 'result', 'langs',
       'temp', 'result_len', 'source_text', 'cui_2_text', 'relation 2',
       'cui_3', 'cui_3_text', 'relation', 'relation_2', 'relation_3', 'cui_4',
       'cui_4_text', 'source_clean', 'cui_2_clean', 'cui_3_clean',
       'cui_4_clean', 'relation_1', 'rel_1_clean', 'rel_2_clean',
       'rel_3_clean', 'combined_entities', 'source_t5', 'target_t5', 'key_id',
       'target_enc'],
      dtype='object')

In [70]:
write_df_to_json(t5_train[['source_t5','target_t5']], 't5_train.json', data_dir = '../data/final/')
write_df_to_json(t5_test[['source_t5','target_t5']], 't5_test.json', data_dir = '../data/final/')
write_df_to_json(t5_val[['source_t5','target_t5']], 't5_val.json', data_dir = '../data/final/')

write_df_to_json(t5_train, 'full_t5_train.json', data_dir = '../data/final/')
write_df_to_json(t5_val, 'full_t5_val.json', data_dir = '../data/final/')
write_df_to_json(t5_test, 'full_t5_test.json', data_dir = '../data/final/')